베이즈 네트워크의 근사 추론(approximate inference). 여기 제공하는 코드는 GitHub aima-python의 코드를 기반으로 일부 수정한 것임.

In [1]:
# 베이즈 네트워크 기본 코드를 bayesnet.py에 저장해뒀음
from bayesnet import *
seed = 23
import random

## 근사 추론 알고리즘 구현

In [2]:
# 무증거 랜덤 샘플링
def prior_sample(bn):
    """
    베이즈 네트워크 bn의 완전 결합 확률 분포에서 무작위 샘플링.
    {변수: 값} 형식의 딕셔너리가 리턴됨.
    """
    event = {}
    for node in bn.nodes:
        event[node.variable] = node.sample(event)
    return event

In [3]:
# 기각 샘플링
def rejection_sampling(X, e, bn, N=10000):
    """
    N개의 샘플을 사용하여 P(X|e)를 추정함.
    N개 샘플이 모두 기각되면 ZeroDivisionError 발생.
    """
    counts = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample = prior_sample(bn)
        if consistent_with(sample, e):
            counts[sample[X]] += 1
    return ProbDist(X, counts)


def consistent_with(event, evidence):
    """event가 evidence와 일관되는가?(증거와 부합하는가?)"""
    return all(evidence.get(k, v) == v for k, v in event.items())

In [4]:
# 가능도 가중치
def likelihood_weighting(X, e, bn, N=10000):
    """
    N개의 샘플을 사용하여 P(X|e)를 추정함.
    """
    W = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample, weight = weighted_sample(bn, e)
        W[sample[X]] += weight
    return ProbDist(X, W)


def weighted_sample(bn, e):
    """
    증거 e와 일관된 이벤트를 bn으로부터 샘플링함.
    이벤트와 가중치(이벤트가 증거에 부합할 가능도)를 리턴함.
    """
    w = 1
    event = dict(e)
    for node in bn.nodes:
        Xi = node.variable
        if Xi in e:
            w *= node.p(e[Xi], event)
        else:
            event[Xi] = node.sample(event)
    return event, w


In [5]:
# Gibbs 샘플링
def gibbs_ask(X, e, bn, N=1000):
    """Gibbs 샘플링."""
    assert X not in e, "Query variable must be distinct from evidence"
    counts = {x: 0 for x in bn.variable_values(X)}
    Z = [var for var in bn.variables if var not in e]
    state = dict(e)
    for Zi in Z:
        state[Zi] = random.choice(bn.variable_values(Zi))
    for j in range(N):
        for Zi in Z:
            state[Zi] = markov_blanket_sample(Zi, state, bn)
            counts[state[X]] += 1
    return ProbDist(X, counts)


def markov_blanket_sample(X, e, bn):
    """
    P(X | mb)에 따라 샘플을 리턴함.
    mb: X의 마르코프 블랭킷에 속한 변수들. 그 값은 이벤트 e에서 취함.
    x의 마르코프 블랭킷: X의 부모들, 자식들, 자식들의 부모들
    """
    Xnode = bn.variable_node(X)
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        ei = extend(e, X, xi)
        Q[xi] = Xnode.p(xi, e) * product(Yj.p(ei[Yj.variable], ei) for Yj in Xnode.children)
    # 불리언 변수로 가정함.
    return probability(Q.normalize()[True])


def product(numbers):
    """numbers에 있는 값들을 모두 곱한 결과를 리턴함. 예: product([2, 3, 10]) == 60"""
    result = 1
    for x in numbers:
        result *= x
    return result

## 근사 추론 예

In [26]:
# 결정론적 노드 o
Aplus = BayesNet([
    ("교수님의수제자", '', 0.05),
    ('선배와친분', "교수님의수제자", {T: 0.8, F: 0.3}),
    ('족보보유', '선배와친분', {T: 0.7, F: 0.1}),
    ('외부대외활동', '교수님의수제자', {T: 0.85, F: 0.1}),
    ('높은아이큐', '', 0.2),
    ('사전지식보유', '외부대외활동 높은아이큐', {(T, T): 0.9, (T, F): 0.75, (F, T): 0.35, (F, F): 0.05}),
    ('체력좋음', '', 0.25),
    ('매주예복습', '체력좋음', {T: 0.6, F: 0.2}),
    ('자료3회독', '매주예복습', {T: 0.9, F: 0.15}),
    ('정시기상', '체력좋음', {T: 0.99, F: 0.90}),
    ('정시출석','정시기상',{T:1, F:0}),
    ('에이플', '족보보유 사전지식보유 자료3회독 정시출석', {(T, T, T, T): 0.97, (T, T, F, T): 0.7, (T, F, T, T): 0.85, (F, T, T, T): 0.7, (T, F, F, T): 0.5, (F, T, F, T): 0.3, (F, F, T, T): 0.4, (F, F, F, T): 0.05, (T, T, T, F): 0.5, (T, T, F, F): 0.4, (T, F, T, F): 0.35, (F, T, T, F): 0.35, (T, F, F, F): 0.2, (F, T, F, F): 0.15, (F, F, T, F): 0.1, (F, F, F, F): 0.03})
])

In [37]:
# 결정론적 노드 o
Aplus3 = BayesNet([
    ("교수님의수제자", '', 0.05),
    ('선배와친분', "교수님의수제자", {T: 0.8, F: 0.3}),
    ('족보보유', '선배와친분', {T: 0.7, F: 0.1}),
    ('외부대외활동', '교수님의수제자', {T: 0.85, F: 0.1}),
    ('높은아이큐', '', 0.2),
    ('사전지식보유', '외부대외활동 높은아이큐', {(T, T): 0.9, (T, F): 0.75, (F, T): 0.35, (F, F): 0.05}),
    ('체력좋음', '', 0.25),
    ('매주예복습', '체력좋음', {T: 0.6, F: 0.2}),
    ('자료3회독', '매주예복습', {T: 0.9, F: 0.15}),
    ('정시출석', '체력좋음', {T: 0.99, F: 0.90}),
    ('에이플', '족보보유 사전지식보유 자료3회독 정시출석', {(T, T, T, T): 0.97, (T, T, F, T): 0.7, (T, F, T, T): 0.85, (F, T, T, T): 0.7, (T, F, F, T): 0.5, (F, T, F, T): 0.3, (F, F, T, T): 0.4, (F, F, F, T): 0.05, (T, T, T, F): 0.5, (T, T, F, F): 0.4, (T, F, T, F): 0.35, (F, T, T, F): 0.35, (T, F, F, F): 0.2, (F, T, F, F): 0.15, (F, F, T, F): 0.1, (F, F, F, F): 0.03})
])

In [39]:
# Gibbs 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정
# 결정론 : .335, 비결정론 : .362
for _ in range(10):
    dist = gibbs_ask('에이플', dict(), Aplus3, 10000)
    print(dist.show_approx())
    print(dist[True])

False: 0.653, True: 0.347
0.3469
False: 0.653, True: 0.347
0.3473909090909091
False: 0.651, True: 0.349
0.3489
False: 0.658, True: 0.342
0.3418
False: 0.651, True: 0.349
0.349
False: 0.677, True: 0.323
0.3233
False: 0.658, True: 0.342
0.3417909090909091
False: 0.646, True: 0.354
0.3536909090909091
False: 0.657, True: 0.343
0.34300909090909093
False: 0.665, True: 0.335
0.3348


In [27]:
N = 1000
all_observations = [prior_sample(Aplus) for x in range(N)]
all_observations[:1]

[{'교수님의수제자': False,
  '선배와친분': True,
  '족보보유': True,
  '외부대외활동': False,
  '높은아이큐': False,
  '사전지식보유': False,
  '체력좋음': False,
  '매주예복습': False,
  '자료3회독': False,
  '정시기상': False,
  '정시출석': False,
  '에이플': True}]

In [28]:
# P(에이플=True)
Aplus_true = [observation for observation in all_observations if observation['에이플'] == True]
answer = len(Aplus_true) / N
print(answer)

0.359


In [29]:
# 다시 샘플링하여 계산
N = 1000
all_observations = [prior_sample(Aplus) for x in range(N)]
Aplus_true = [observation for observation in all_observations if observation['정시출석'] == True]
answer = len(Aplus_true) / N
print(answer)

0.915


In [30]:
# P(교수님의수제자=True | 에이플=True)
pp_and_Aplus = [observation for observation in Aplus_true if observation['교수님의수제자'] == True]
answer = len(pp_and_Aplus) / len(Aplus_true)
print(answer)

0.05245901639344262


In [31]:
# 기각 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정
#random.seed(seed)
dist = rejection_sampling('족보보유', dict(에이플=True, 높은아이큐=True), Aplus, 10000)
print(dist.show_approx())
print(dist[True])

False: 0.467, True: 0.533
0.5331664580725908


In [32]:
weighted_sample(Aplus, dict(에이플=True))

({'에이플': True,
  '교수님의수제자': False,
  '선배와친분': False,
  '족보보유': False,
  '외부대외활동': False,
  '높은아이큐': False,
  '사전지식보유': False,
  '체력좋음': True,
  '매주예복습': True,
  '자료3회독': True,
  '정시기상': True,
  '정시출석': True},
 0.4)

In [33]:
# 가능도 가중치를 사용하여 P(에이플=True | 교수님의수제자=True) 추정
#random.seed(seed)
dist = likelihood_weighting('에이플', dict(출석=True), Aplus, 1000)
print(dist.show_approx())
print(dist[True])

False: 0.618, True: 0.382
0.382


In [34]:
# Gibbs 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정
# 결정론 : .335, 비결정론 : .362
for _ in range(10):
    dist = gibbs_ask('에이플', dict(), Aplus, 10000)
    print(dist.show_approx())
    print(dist[True])

False: 0.862, True: 0.138
0.13819166666666666
False: 0.847, True: 0.153
0.1533
False: 0.866, True: 0.134
0.1344
False: 0.63, True: 0.369
0.3695
False: 0.64, True: 0.36
0.3595083333333333
False: 0.627, True: 0.373
0.3732
False: 0.634, True: 0.366
0.3656916666666667
False: 0.853, True: 0.147
0.1466
False: 0.857, True: 0.143
0.1435
False: 0.862, True: 0.138
0.13789166666666666


In [35]:
# 결정론적인 노드 x
Aplus2 = BayesNet([
    ("교수님의수제자", '', 0.05),
    ('선배와친분', "교수님의수제자", {T: 0.8, F: 0.3}),
    ('족보보유', '선배와친분', {T: 0.7, F: 0.1}),
    ('외부대외활동', '교수님의수제자', {T: 0.85, F: 0.1}),
    ('높은아이큐', '', 0.2),
    ('사전지식보유', '외부대외활동 높은아이큐', {(T, T): 0.9, (T, F): 0.75, (F, T): 0.35, (F, F): 0.05}),
    ('체력좋음', '', 0.25),
    ('매주예복습', '체력좋음', {T: 0.6, F: 0.2}),
    ('자료3회독', '매주예복습', {T: 0.9, F: 0.15}),
    ('에이플', '족보보유 사전지식보유 자료3회독', {(T, T, T): 0.97, (T, T, F): 0.7, (T, F, T): 0.85, (F, T, T): 0.7, (T, F, F): 0.5, (F, T, F): 0.3, (F, F, T): 0.4, (F, F, F): 0.05})
])

In [36]:
# Gibbs 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정 안고장남
# 결정론 : .335, 비결정론 : .362
for _ in range(10):
    dist = gibbs_ask('에이플', dict(), Aplus2, 10000)
    print(dist.show_approx())
    print(dist[True])

False: 0.645, True: 0.355
0.35451
False: 0.628, True: 0.372
0.37161
False: 0.629, True: 0.371
0.37069
False: 0.638, True: 0.362
0.362
False: 0.646, True: 0.354
0.35369
False: 0.632, True: 0.368
0.36761
False: 0.656, True: 0.344
0.34431
False: 0.631, True: 0.369
0.3691
False: 0.627, True: 0.373
0.37309
False: 0.629, True: 0.371
0.3711
